In [182]:
import matplotlib
%matplotlib inline
from collections import deque
import time

## Check Function

In [237]:
def test_wheather_the_same(solution_a, solution_b):
    '''
    params:solution_a, solution_b: lists
    check if the two permutation solutions for tour are actually the same
    '''
    dequeue_a = deque(solution_a)
    dequeue_b = deque(solution_b)
    while(dequeue_a[0]!=dequeue_b[0]):
        # circular shift from left to right
        dequeue_a.rotate()
    # check from left to right
    if dequeue_a == dequeue_b:
        return True
    else:
        dequeue_a.reverse()
        dequeue_a.rotate()
        if dequeue_a == dequeue_b:
            return True
        else:
            return False

def count_neighbors(neighbors):
    '''
    params: neighbors: a list of solutions probably with duplications
    output: actual number of neighbors
    '''
    total_count = len(neighbors) - 1
    for idx, solution_a in enumerate(neighbors):
        for solution_b in neighbors[idx+1:]:
            
            if test_wheather_the_same(solution_a, solution_b):
                #print(solution_a, solution_b)
                #print("same")
                total_count -= 1
                break
    return total_count

## Adjacent Swap

In [89]:
def ad_swap(var_size):
    init_solution = list(range(var_size))
    neighbors = list()
    neighbors.append(init_solution)
    for idx in range(var_size):
        newsolutoin = list(init_solution)
        tmp = newsolutoin[idx-1]
        newsolutoin[idx-1] = newsolutoin[idx]
        newsolutoin[idx] = tmp
        neighbors.append(newsolutoin)
    return neighbors

In [220]:
for i in range(100):
    if i != count_neighbors(ad_swap(i)):
        print(i)

[0] [0]
same
1
[0, 1] [1, 0]
same
[1, 0] [1, 0]
same
2
[0, 1, 2] [2, 1, 0]
same
[2, 1, 0] [1, 0, 2]
same
[1, 0, 2] [0, 2, 1]
same
3
[3, 1, 2, 0] [0, 2, 1, 3]
same
[1, 0, 2, 3] [0, 1, 3, 2]
same
4


## Arbitary Swap

In [86]:
def arbitary_swap(var_size):
    init_solution = list(range(var_size))
    neighbors = list()
    neighbors.append(init_solution)
    for idx1 in range(var_size):
        for idx2 in range(idx1 + 1, var_size):
            newsolutoin = list(init_solution)
            tmp = newsolutoin[idx1]
            newsolutoin[idx1] = newsolutoin[idx2]
            newsolutoin[idx2] = tmp
            neighbors.append(newsolutoin)
    return neighbors

n >= 6 size = nC2

In [223]:
for i in range(40):
    if i*(i-1)/2 != count_neighbors(arbitary_swap(i)):
        print(i)

[0, 1] [1, 0]
same
2
[0, 1, 2] [1, 0, 2]
same
[1, 0, 2] [2, 1, 0]
same
[2, 1, 0] [0, 2, 1]
same
3
[0, 1, 2, 3] [2, 1, 0, 3]
same
[1, 0, 2, 3] [0, 1, 3, 2]
same
[2, 1, 0, 3] [0, 3, 2, 1]
same
[3, 1, 2, 0] [0, 2, 1, 3]
same
4
[1, 0, 2, 3, 4] [0, 1, 4, 3, 2]
same
[2, 1, 0, 3, 4] [0, 1, 2, 4, 3]
same
[3, 1, 2, 0, 4] [0, 2, 1, 3, 4]
same
[4, 1, 2, 3, 0] [0, 3, 2, 1, 4]
same
[0, 4, 2, 3, 1] [0, 1, 3, 2, 4]
same
5
[2, 1, 0, 3, 4, 5] [0, 1, 2, 5, 4, 3]
same
[4, 1, 2, 3, 0, 5] [0, 3, 2, 1, 4, 5]
same
[0, 5, 2, 3, 4, 1] [0, 1, 4, 3, 2, 5]
same
6


## Insertion

In [231]:
def insertion(var_size):
    init_solution = list(range(var_size))
    neighbors = list()
    neighbors.append(init_solution)
    for idx1 in range(var_size):
        for idx2 in range(var_size):
            newsolutoin = list(init_solution)
            del newsolutoin[idx1]
            if idx2 < idx1:
                newsolutoin.insert(idx2, idx1)
            elif idx2 > idx1:
                newsolutoin.insert(idx2, idx1)
            else:
                continue
            neighbors.append(newsolutoin)
    # print(neighbors)
    return neighbors

In [240]:
for i in range(40):
    if i**2 - 3*i != count_neighbors(insertion(i)):
        print(i)

1
2
4


## Two-edge Inversion

In [258]:
def two_opt(var_size):
    init_solution = list(range(var_size))
    neighbors = list()
    neighbors.append(init_solution)
    for idx1 in range(1, var_size-1):
        for idx2 in range(idx1+1, var_size):
            newsolutoin = list(init_solution)
            newsolutoin[idx1:idx2+1] = newsolutoin[idx2:idx1-1:-1]
            neighbors.append(newsolutoin)
    #print(neighbors)
    return neighbors

In [259]:
for i in range(40):
    if (i**2 - 3*i)/2 != count_neighbors(two_opt(i)):
        print(i)

1
2


# Local search Neighborhood Structure Comparisom
## Greedy Algorithm

Reference https://github.com/dmishin/tsp-solver

In [461]:
from random import shuffle, randrange, choice, randint, randrange
from itertools import permutations
import numpy as np
cities = np.loadtxt('TSP100.txt',skiprows=1)
def calc_length(cities, path):
    length = 0
    for i in range(len(path)):
        square = dist_squared(cities[path[i-1]], cities[path[i]])
        #print(square)
        length += square
    return length

import math

def dist_squared(c1, c2):
    t1 = c2[0] - c1[0]
    t2 = c2[1] - c1[1]
    return math.sqrt(t1**2 + t2**2)

In [457]:
def greedy_algorithm(cities):
    best_order = []
    best_length = float('inf')

    for i_start, start in enumerate(cities):
        order = [i_start]
        length = 0
        i_next, next, dist = get_closest(start, cities, order)
        length += dist
        order.append(i_next)

        while len(order) < cities.shape[0]:
            i_next, next, dist = get_closest(next, cities, order)
            length += dist
            order.append(i_next)
        #print(order)
        if length < best_length:
            best_length = length
            best_order = order
    return best_order, best_length

In [458]:
start = time.time()
result1 = greedy_algorithm(cities)
print(time.time() - start)

0.8597748279571533


In [459]:
result1[1]

4176944.4222224886

## Swap-3 random

In [460]:
def swap3_random_algorithm(cities):
    order =  list(range(cities.shape[0]))
    shuffle(order)
    length = calc_length(cities, order)
    start = time.time()
    changed = True
    while changed:
        changed = False
        for a in range(cities.shape[0]):
            for b in range(a+1, cities.shape[0] - 1):
                c =  randint(b+1, cities.shape[0]-1)
                a, b, c = choice(list(permutations((a,b,c))))
                new_order = order[:a] + order[a:b][::-1] + order[b:c][::-1] + order[c:]
                new_length = calc_length(cities, new_order)
                if new_length < length:
                    length = new_length
                    order = new_order
                    changed = True
    return order, length

In [216]:
time3 = list()
result3 = list()
for _ in range(6):
    start = time.time()
    result3.append(swap3_random_algorithm(cities))
    time3.append(time.time() - start)

In [266]:
for val in time3:
    print(val)

18.29692578315735
13.820849418640137
18.516886949539185
24.232131242752075
39.02716064453125
23.27146029472351


In [262]:
for val in result3:
    print(val[1])

37416110.4189
40408332.7233
36137222.3978
46413332.6233
35644999.8733
25830555.3711


In [375]:
MAX_EVALUATION = 10000

## Swap-3

In [473]:
def swap3_algorithm(cities):
    order =  list(range(cities.shape[0]))
    shuffle(order)
    length = calc_length(cities, order)
    start = time.time()
    changed = True
    iter_count = 0
    while changed and iter_count < MAX_EVALUATION:
        #changed = False
        for a in range(cities.shape[0]):
            for b in range(a+1, cities.shape[0] - 1):
                c = randint(b+1, cities.shape[0]-1)
                for a,b,c in permutations((a,b,c)):
                    iter_count += 1
                    new_order = order[:a] + order[a:b][::-1] + order[b:c][::-1] + order[c:]
                    new_length = calc_length(cities, new_order)
                    if new_length < length:
                        length = new_length
                        order = new_order
                        #changed = True
    return order, length, iter_count

In [475]:
time5 = list()
result5 = list()
for _ in range(10):
    start = time.time()
    result5.append(list(swap3_algorithm(cities)[1:])+[time.time() - start])
cost, iterations, time_used = 0, 0, 0
for sample in result5:
    cost += sample[0]
    iterations += sample[1]
    time_used += sample[2]
print(cost/10, iterations/10, time_used/10)

31508.386792083107 29106.0 4.114355516433716


In [352]:
def swap2_adjacent_algorithm(cities):
    # init
    best_order = []
    best_length = float('inf')
    order =  list(range(cities.shape[0]))
    shuffle(order)
    best_order = order
    best_length = calc_length(cities, order)
    # print(best_length)
    
    iter_count = 0
    changed = True
    while iter_count < MAX_EVALUATION and changed:
        changed = False
        for idx in range(cities.shape[0]-1):
            iter_count += 1
            d1_1 = dist_squared(cities[best_order[idx]], cities[best_order[idx+1]])
            d1_2 = dist_squared(cities[best_order[idx-1]], cities[best_order[idx+1]])
            d2_1 = dist_squared(cities[best_order[idx-1]], cities[best_order[idx-2]])
            d2_2 = dist_squared(cities[best_order[idx]], cities[best_order[idx-2]])
            # print(dist_squared(cities[best_order[idx-1]], cities[best_order[idx]]))
            if d1_1+d2_1 > d1_2 + d2_2:
                new_order = list(best_order)
                tmp = new_order[idx-1]
                new_order[idx-1] = new_order[idx]
                new_order[idx] = tmp
                best_order = new_order
                best_length = best_length + d1_2 + d2_2 - d1_1 - d2_1
                # print(d1_2 + d2_2 - d1_1 - d2_1, idx)
                #print(dist_squared(cities[best_order[idx-1]], cities[best_order[idx]]))
                changed = True
                break
    return best_order, best_length, iter_count

In [364]:
tour,cost,iteration = swap2_adjacent_algorithm(cities)

In [427]:
time1 = list()
result1 = list()
for _ in range(10):
    start = time.time()
    result1.append(list(swap2_adjacent_algorithm(cities)[1:])+[time.time() - start])
cost, iterations, time_used = 0, 0, 0
for sample in result1:
    cost += sample[0]
    iterations += sample[1]
    time_used += sample[2]
print(cost/10, iterations/10, time_used/10)

79139.12594304237 3210.2 0.01595611572265625


In [420]:
def swap2_arbitrary_algorithm(cities):
    # init
    N = cities.shape[0]
    best_order = []
    best_length = float('inf')
    order =  list(range(N))
    shuffle(order)
    best_order = order
    best_length = calc_length(cities, order)
    # print(best_length)
    iter_count = 0
    changed = True
    while iter_count < MAX_EVALUATION and changed:
        changed = False
        for idx in range(cities.shape[0]-1):
            iter_count += 1
            d1_1 = dist_squared(cities[best_order[idx]], cities[best_order[idx+1]])
            d1_2 = dist_squared(cities[best_order[idx-1]], cities[best_order[idx+1]])
            d2_1 = dist_squared(cities[best_order[idx-1]], cities[best_order[idx-2]])
            d2_2 = dist_squared(cities[best_order[idx]], cities[best_order[idx-2]])
            # print(dist_squared(cities[best_order[idx-1]], cities[best_order[idx]]))
            if d1_1+d2_1 > d1_2 + d2_2:
                new_order = list(best_order)
                tmp = new_order[idx-1]
                new_order[idx-1] = new_order[idx]
                new_order[idx] = tmp
                best_order = new_order
                best_length = best_length + d1_2 + d2_2 - d1_1 - d2_1
                # print(d1_2 + d2_2 - d1_1 - d2_1, idx)
                #print(dist_squared(cities[best_order[idx-1]], cities[best_order[idx]]))
                changed = True
                break
        for idx1 in range(N):
            for idx2 in range(idx1 + 2, N):
                iter_count += 1
                # swap idx1 and idx2
                #print(idx1, idx2, (idx1+1) % N, (idx2+1) % N)
                d1_1 = dist_squared(cities[best_order[idx1]], cities[best_order[idx1-1]]) + \
                    dist_squared(cities[best_order[idx1]], cities[best_order[(idx1+1) % N]])
                d1_2 = dist_squared(cities[best_order[idx2]], cities[best_order[idx1-1]]) + \
                    dist_squared(cities[best_order[idx2]], cities[best_order[(idx1+1) % N]])
                d2_1 = dist_squared(cities[best_order[idx2]], cities[best_order[idx2-1]]) + \
                    dist_squared(cities[best_order[idx2]], cities[best_order[(idx2+1) % N]])
                d2_2 = dist_squared(cities[best_order[idx1]], cities[best_order[idx2-1]]) + \
                    dist_squared(cities[best_order[idx1]], cities[best_order[(idx2+1) % N]])
                if d1_1+d2_1 > d1_2 + d2_2:
                    newsolutoin = list(best_order)
                    tmp = newsolutoin[idx1]
                    newsolutoin[idx1] = newsolutoin[idx2]
                    newsolutoin[idx2] = tmp
                    best_order = newsolutoin
                    best_length = best_length + d1_2 + d2_2 - d1_1 - d2_1
                    #print(d1_2,  d2_2,d1_1 , d2_1, iter_count)
                    changed = True
                    break
    return best_order, best_length, iter_count

In [421]:
tour,cost,iteration = swap2_arbitrary_algorithm(cities)

In [416]:
time2 = list()
result2 = list()
for _ in range(10):
    start = time.time()
    result2.append(list(swap2_arbitrary_algorithm(cities)[1:])+[time.time() - start])
cost, iterations, time_used = 0, 0, 0
for sample in result2:
    cost += sample[0]
    iterations += sample[1]
    time_used += sample[2]
print(cost/10, iterations/10, time_used/10)

30779.753017996038 10871.0 0.10801308155059815


In [439]:
def insertion_algorithm(cities):
    # init
    N = cities.shape[0]
    best_order = []
    best_length = float('inf')
    order =  list(range(N))
    shuffle(order)
    best_order = order
    best_length = calc_length(cities, order)
    # print(best_length)
    iter_count = 0
    changed = True
    while iter_count < MAX_EVALUATION and changed:
        changed = False
        for idx1 in range(N):
            for idx2 in range(N):
                # insert idx1 before idx2
                if idx2 < idx1 or idx2 > idx1:
                    iter_count += 1
                    d1_1 = dist_squared(cities[best_order[idx1]], cities[best_order[(idx1+1)%N]]) +\
                        dist_squared(cities[best_order[idx1]], cities[best_order[idx1-1]])
                    d2_1 = dist_squared(cities[best_order[idx2]], cities[best_order[idx2-1]])
                    d1_2 = dist_squared(cities[best_order[idx1]], cities[best_order[idx2]]) +\
                        dist_squared(cities[best_order[idx1]], cities[best_order[idx2-1]])
                    d2_2 = dist_squared(cities[best_order[idx1-1]], cities[best_order[(idx1+1)%N]])
                    if d1_1+d2_1 > d1_2 + d2_2:
                        new_order = list(best_order)
                        del new_order[idx1]
                        new_order.insert(idx2, best_order[idx1])
                        best_order = new_order
                        best_length = best_length + d1_2 + d2_2 - d1_1 - d2_1
                        # print(d1_2 + d2_2 - d1_1 - d2_1, idx)
                        #print(dist_squared(cities[best_order[idx-1]], cities[best_order[idx]]))
                        changed = True
                        break
    return best_order, best_length, iter_count

In [440]:
tour,cost,iteration = insertion_algorithm(cities)

In [441]:
calc_length(cities, tour)

34260.28090926645

In [442]:
time3 = list()
result3 = list()
for _ in range(10):
    start = time.time()
    result3.append(list(swap2_arbitrary_algorithm(cities)[1:])+[time.time() - start])
cost, iterations, time_used = 0, 0, 0
for sample in result3:
    cost += sample[0]
    iterations += sample[1]
    time_used += sample[2]
print(cost/10, iterations/10, time_used/10)

30581.257082755976 11322.2 0.1110116720199585


In [445]:
def two_opt_algorithm(cities):
    # init
    N = cities.shape[0]
    best_order = []
    best_length = float('inf')
    order =  list(range(N))
    shuffle(order)
    best_order = order
    best_length = calc_length(cities, order)
    # print(best_length)
    iter_count = 0
    changed = True
    while iter_count < MAX_EVALUATION and changed:
        changed = False
        for idx1 in range(1, N-1):
            for idx2 in range(idx1+1, N):
                iter_count += 1
                d1_1 = dist_squared(cities[best_order[idx1-1]], cities[best_order[idx1]])
                d2_1 = dist_squared(cities[best_order[idx2]], cities[best_order[(idx2+1)%N]])
                d1_2 = dist_squared(cities[best_order[idx1-1]], cities[best_order[idx2]])
                d2_2 = dist_squared(cities[best_order[idx1]], cities[best_order[(idx2+1)%N]])
                if d1_1+d2_1 > d1_2 + d2_2:
                    newsolutoin = list(best_order)
                    newsolutoin[idx1:idx2+1] = newsolutoin[idx2:idx1-1:-1]
                    best_order = newsolutoin
                    best_length = best_length + d1_2 + d2_2 - d1_1 - d2_1
                    changed = True
                    break
    return best_order, best_length, iter_count

In [446]:
tour,cost,iteration = two_opt_algorithm(cities)

In [449]:
time4 = list()
result4 = list()
for _ in range(10):
    start = time.time()
    result4.append(list(swap2_arbitrary_algorithm(cities)[1:])+[time.time() - start])
cost, iterations, time_used = 0, 0, 0
for sample in result4:
    cost += sample[0]
    iterations += sample[1]
    time_used += sample[2]
print(cost/10, iterations/10, time_used/10)

31382.33400950809 10732.0 0.107138991355896


In [469]:
def three_opt_algorithm(cities):
    # init
    N = cities.shape[0]
    best_order = []
    best_length = float('inf')
    order =  list(range(N))
    shuffle(order)
    best_order = order
    best_length = calc_length(cities, order)
    # print(best_length)
    iter_count = 0
    changed = True
    while iter_count < MAX_EVALUATION and changed:
        changed = False
        for a in range(N):
            for b in range(a+1, N-1):
                c = randint(b+1, N-1)
                d,e,f = a-1, b-1, c-1
                pass
            #ref https://stackoverflow.com/questions/21205261/3-opt-local-search-for-tsp
            # need to be finished
    return best_order, best_length, iter_count

In [ ]:
time5 = list()
result5 = list()
for _ in range(10):
    start = time.time()
    result5.append(list(three_opt_algorithm(cities)[1:])+[time.time() - start])
cost, iterations, time_used = 0, 0, 0
for sample in result5:
    cost += sample[0]
    iterations += sample[1]
    time_used += sample[2]
print(cost/10, iterations/10, time_used/10)

In [468]:
for x in permutations((2,5,3)):
    print(x)

(2, 5, 3)
(2, 3, 5)
(5, 2, 3)
(5, 3, 2)
(3, 2, 5)
(3, 5, 2)
